In [65]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import requests
from newspaper import Article
from newspaper import Config
from urllib.request import urlopen
import re
from collections import Counter
c = Counter()  
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'
config = Config()
config.browser_user_agent = user_agent
findword = 'biden'    #What word are you looking for
news_url="https://news.google.com/news/rss"

In [66]:
Client=urlopen(news_url)
xml_page=Client.read()
Client.close()
soup_page=soup(xml_page,"xml")
news_list=soup_page.findAll("item")

def word_search(findword):
    search1 = rf'(?i)\b{findword}\b'      #r'(?i)\bbob\b'  # matches bob, Bob
    contents = news.title.text
    count = sum(1 for match in re.finditer(search1, contents))
    return count

In [67]:
article_count = 0
d = {}
i=0
for news in news_list:
    if (word_search(findword)) > 0:     
        article_count = article_count + 1
        d[i]  = news.pubDate.text, news.title.text, news.link.text
        i=i+1
print('Total number of articles:', len(news_list))
print('Number of articles with search term:', article_count)
df = pd.DataFrame(data=d)

Total number of articles: 38
Number of articles with search term: 2


In [68]:
df= df.T
df.columns = ['Timestamp','Title','URL']
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [69]:
for i in range(len(df['URL'])):
    try:
        url = df['URL'][i].strip()
        article = Article(url, config=config)
        article.download()
        article.parse()
        #print(article.title + 2*('\n') + ('-----------\n') + article.text) + ('---------------------------\n')
        content = article.text
        df.loc[df.index[i], 'Content'] = str(content)
    except:
        print("ERROR")
df

ERROR


,Timestamp,Title,URL,Content
0,2021-03-31 15:32:00+00:00,Here’s how Biden’s $2 trillion infrastructure ...,https://news.google.com/__i/rss/rd/articles/CB...,Vice President Kamala Harris (2-L) and Special...
1,2021-03-31 14:36:00+00:00,Dow Jones Rallies As Biden Unveils $2 Tril. In...,https://news.google.com/__i/rss/rd/articles/CB...,NaN


In [70]:
import spacy
nlp = spacy.load("en_core_web_lg")
from spacy import displacy

In [71]:
for i in range(len(df['URL'])):
    text = str(df['Content'][i])
    doc = nlp(text)
    entities=[]
    for ent in doc.ents:
        if ent.label_=="PERSON" and ent.text not in entities:
            entities.append(str(ent.text))
    df.loc[df.index[i], 'Entities'] = str(entities)

In [72]:
df

,Timestamp,Title,URL,Content,Entities
0,2021-03-31 15:32:00+00:00,Here’s how Biden’s $2 trillion infrastructure ...,https://news.google.com/__i/rss/rd/articles/CB...,Vice President Kamala Harris (2-L) and Special...,"['Kamala Harris', 'John Kerry', 'Joe Biden', '..."
1,2021-03-31 14:36:00+00:00,Dow Jones Rallies As Biden Unveils $2 Tril. In...,https://news.google.com/__i/rss/rd/articles/CB...,NaN,[]
